# Coursework
## Programming in Python for Business Analytics
#### Group 19

#### Machine Learning Models

In [3]:
# Libraries:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

In [20]:
dataset = pd.read_csv('data_py.csv')
tasks = pd.read_csv('/Users/alexander/Documents/GitHub/PythonCoursework/data/tasks.csv')
suppliers = pd.read_csv('/Users/alexander/Documents/GitHub/PythonCoursework/data/suppliers.csv')
cost = pd.read_csv('/Users/alexander/Documents/GitHub/PythonCoursework/data/cost.csv.zip')

In [7]:
# Splitting into test and train datasets, manually to preserve the groups
X = dataset.drop(['Task ID','Supplier ID','Cost'], axis='columns')
y = dataset['Cost']
Groups = dataset['Task ID']

all_tasks = Groups.unique()
random.seed(42)
TestGroup = np.random.choice(all_tasks, size=20, replace=False)

test_loc = dataset['Task ID'].isin(list(TestGroup))

X_test = X[test_loc]
y_test = y[test_loc]
X_train = X[~test_loc]
y_train = y[~test_loc]

In [13]:
# 3.3 Fitting a ridge regression model
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)
ridge.score(X_test,y_test)
y_pred = ridge.predict(X_test)

In [15]:
# 3.4 Grouping by tasks and selecting a supplier for each test task
# using the error formula provided calculate difference between true supplier and predicted supplier cost

# Getting Task IDs for our predictions
ids = dataset['Task ID'][test_loc].reset_index(drop=True)
y_pred = pd.Series(y_pred, name="Predicted Cost")

y_pred_ids = pd.merge(ids, y_pred, right_index = True, left_index = True)
pred_best_suppliers = y_pred_ids.groupby('Task ID')['Predicted Cost'].min()

In [22]:
# Getting the true cheapest suppliers for our test set
true_best_suppliers = cost.groupby('Task ID')['Cost'].min()

test_best_suppliers = true_best_suppliers[true_best_suppliers.index.isin(TestGroup)]

In [24]:
# Defining functions to calculate errors and rmse
def error_calc(true_min_costs, predicted_min_cost):
   error = true_min_costs - predicted_min_cost
   return error

def rmse_calc(error_array):
    squared_errors = error_array*error_array
    rss = np.sum(squared_errors)
    value = np.sqrt(rss/len(error_array))
    return value

ridge_error = error_calc(test_best_suppliers, pred_best_suppliers)
ridge_rmse = rmse_calc(ridge_error)

In [26]:
# Results for an initial ridge model
print(ridge_error)
print(ridge_rmse)

Task ID
T107    0.013728
T108   -0.022155
T113   -0.031174
T127   -0.018289
T14    -0.051290
T16    -0.045721
T28    -0.038868
T34    -0.096904
T39    -0.005387
T40    -0.033426
T45    -0.017008
T6     -0.043307
T8     -0.033803
T83    -0.006378
T85    -0.036355
T87    -0.039659
T88    -0.025602
T93     0.026752
T98    -0.001158
T99     0.004425
dtype: float64
0.0363241273831713


In [28]:
# 4 Performing cross validation
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.metrics import make_scorer

train_tasks = dataset['Task ID'][~test_loc]

ridge_cv = Ridge(alpha=0.5)
logo = LeaveOneGroupOut()

In [30]:
# Defining our scoring function:
# Since the test set for each fold of the cross validation only contains one group
# we can just find the minimums of each array, then compute rmse from that

def error_scoring(all_trues, all_preds):
    error = all_trues.min() - all_preds.min()
    return error
error_scorer = make_scorer(error_scoring)

scores = cross_val_score(ridge_cv, X_train, y_train, cv=logo, groups=train_tasks, scoring=error_scorer)
cv_rmse = rmse_calc(scores)